In [180]:
# %pip install peft evaluate transformers Levenshtein ipywidgets
# %pip install protobuf==3.20.3
# !rm -rf /kaggle/working/cache
# !rm -rf /kaggle/working/outputs

In [181]:
# X

import os
os.environ["TRANSFORMERS_DISABLE_CHAT_TEMPLATES"] = "1"
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
os.environ["TRANSFORMERS_NO_ADDITIONAL_CHAT_TEMPLATES"] = "1"

In [182]:
import random
from datasets import load_dataset, load_from_disk
from transformers import CanineTokenizer
from peft import LoraConfig, TaskType, get_peft_model
import re
import string
from collections import Counter
import numpy as np
import Levenshtein

from transformers import TrainingArguments, Trainer, TrainerCallback
import json
from huggingface_hub import HfApi, notebook_login, whoami

In [183]:
# notebook_login()
# whoami()

In [184]:
from transformers import CanineTokenizer, CanineForQuestionAnswering
import torch
model_name = 'google/canine-s'
device = torch.device("cuda" if torch.cuda.is_available() else "mps")

tokenizer = CanineTokenizer.from_pretrained(model_name, use_fast=False, trust_remote_code=False)
model = CanineForQuestionAnswering.from_pretrained(model_name, trust_remote_code=False)

Some weights of CanineForQuestionAnswering were not initialized from the model checkpoint at google/canine-s and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [185]:
# filter out impossible questions
def filter_function(example):
    return not example['is_impossible']

In [186]:
uqa_dataset = load_dataset("uqa/UQA")

# filtering
uqa_dataset_filtered = uqa_dataset.filter(filter_function)

# trying the full dataset
uqa_train = uqa_dataset_filtered["train"].shuffle(seed=42)
uqa_val = uqa_dataset_filtered["validation"].shuffle(seed=42)

uqa_train = uqa_dataset["train"].shuffle(seed=42).select(range(20000))
uqa_val = uqa_dataset["validation"].shuffle(seed=42).select(range(5000))

print(f"📊 Dataset after filtering:")
print(f"   Original train size: {len(uqa_dataset['train']):,}")
print(f"   Filtered train size: {len(uqa_dataset_filtered['train']):,}")
print(f"   Using for training: {len(uqa_train):,}")
print(f"   Validation size: {len(uqa_val):,}")

📊 Dataset after filtering:
   Original train size: 124,745
   Filtered train size: 83,018
   Using for training: 20,000
   Validation size: 5,000


In [187]:
# Check character-token alignment
ex = uqa_train[444]
context = ex["context"]
context_tokens = tokenizer.encode(ex["context"], add_special_tokens=False)

print(f"Context length (characters): {len(context)}")
print(f"Context length (tokens): {len(context_tokens)}")
print(f"1:1 mapping: {len(context) == len(context_tokens)}")

Context length (characters): 749
Context length (tokens): 749
1:1 mapping: True


In [188]:
# Explore raw UQA dataset structure
print("="*80)
print("UQA DATASET STRUCTURE")
print("="*80)
print(f"Training set size: {len(uqa_train):,} examples")
print(f"Validation set size: {len(uqa_val):,} examples")
print(f"\nDataset columns: {uqa_train.column_names}")
print("\n" + "="*80)

# Show a few examples
print("\n📝 EXAMPLE 1 - Question with Answer")
print("="*80)
ex1 = uqa_train[0]
print(f"Question: {ex1['question']}")
print(f"\nContext (first 300 chars): {ex1['context'][:300]}...")
print(f"\nAnswer: '{ex1['answer']}'")
print(f"Answer starts at character position: {ex1['answer_start']}")

# Verify the answer extraction
if ex1['answer_start'] != -1:
    extracted = ex1['context'][ex1['answer_start']:ex1['answer_start']+len(ex1['answer'])]
    print(f"✓ Extracted from context: '{extracted}'")
    print(f"✓ Match: {extracted == ex1['answer']}")

print("\n" + "="*80)
print("\n📝 EXAMPLE 2 - Another Question")
print("="*80)
ex2 = uqa_train[100]
print(f"Question: {ex2['question']}")
print(f"\nContext length: {len(ex2['context'])} characters")
print(f"Answer: '{ex2['answer']}'")
print(f"Answer starts at position: {ex2['answer_start']}")

# Show answer in context
if ex2['answer_start'] != -1:
    start = max(0, ex2['answer_start'] - 50)
    end = min(len(ex2['context']), ex2['answer_start'] + len(ex2['answer']) + 50)
    context_snippet = ex2['context'][start:end]
    answer_pos = ex2['answer_start'] - start
    print(f"\nContext around answer:")
    print(f"...{context_snippet}...")
    print(f"    {' '*answer_pos}{'~'*len(ex2['answer'])} (answer here)")

print("\n" + "="*80)
print("\n📊 DATASET STATISTICS")
print("="*80)

# Compute some basic statistics
import numpy as np
question_lengths = [len(ex['question']) for ex in uqa_train.select(range(1000))]
context_lengths = [len(ex['context']) for ex in uqa_train.select(range(1000))]
answer_lengths = [len(ex['answer']) if ex['answer'] else 0 for ex in uqa_train.select(range(1000))]
has_answer = [ex['answer_start'] != -1 for ex in uqa_train.select(range(1000))]

print(f"Question length (chars): mean={np.mean(question_lengths):.1f}, max={np.max(question_lengths)}")
print(f"Context length (chars): mean={np.mean(context_lengths):.1f}, max={np.max(context_lengths)}")
print(f"Answer length (chars): mean={np.mean(answer_lengths):.1f}, max={np.max(answer_lengths)}")
print(f"Questions with answers: {sum(has_answer)/len(has_answer)*100:.1f}%")
print(f"Questions without answers: {(1-sum(has_answer)/len(has_answer))*100:.1f}%")

UQA DATASET STRUCTURE
Training set size: 20,000 examples
Validation set size: 5,000 examples

Dataset columns: ['id', 'title', 'context', 'question', 'is_impossible', 'answer', 'answer_start']


📝 EXAMPLE 1 - Question with Answer
Question: جدید ہوائی جہازوں کی تعمیر میں کیا استعمال کیا جاتا تھا؟

Context (first 300 chars): 1906 میں ، الفریڈ وِلم نے بارش سے سخت ہونے والے مرکب دریافت کیے۔ بارش سے سخت ہونے والے مرکب ، جیسے ایلومینیم ، ٹائٹینیم اور تانبے کے کچھ مرکب ، گرمی سے علاج کرنے والے مرکب ہیں جو ٹھنڈا ہونے پر نرم ہوجاتے ہیں (جلدی سے ٹھنڈا ہوجاتے ہیں) ، اور پھر وقت کے ساتھ سخت ہوجاتے ہیں۔ ایلومینیم ، تانبے اور میگنی...

Answer: 'ڈورالومین'
Answer starts at character position: 503
✓ Extracted from context: 'ڈورالومین'
✓ Match: True


📝 EXAMPLE 2 - Another Question
Question: کون سا منگ شہزادہ نانجنگ میں تخت نشین ہوا تھا؟

Context length: 447 characters
Answer: 'جو یو سونگ'
Answer starts at position: 262

Context around answer:
...الی چنگ خاندان کے ہاتھوں گرنے کے بعد ، منگ شہزادہ جو یو 

---

## Updated preprocessors!

Previously, we tried to apply the same approach we used in TYDIQA on UQA, the problem was the preprocessors were aligning the answer spans in units of **byte-level spans** instead of **character-level spans**. The calculations were adding byte-level offsets to the answer lengths, and since Urdu characters may be quantified in multiple bytes, the model was being fed the wrong spans -> GIGO!

We are now testing an updated preprocessor

In [189]:
"""
FIXED preprocessing function for UQA with CANINE-S.
TyDiQA-style preprocessor adapted for UQA character offsets.

Key fixes applied:
1. Uses character-level offsets (UQA native format, no byte conversion needed)
2. Fixed boundary check: uses `<` instead of `<=` for chunk_end
3. Calculates gold_char_end as inclusive (answer_start + len(answer) - 1)
4. Dynamic cls_index for no-answer cases
5. Simplified context_offset calculation

This preprocessor passed all 200 real-world UQA examples in testing.
"""

MAX_SEQ_LENGTH = 384
DOC_STRIDE = 64  # Using TyDiQA's value for proven results

def preprocess_uqa(examples, tokenizer, max_length=MAX_SEQ_LENGTH, doc_stride=DOC_STRIDE, model_obj=None, indices=None):
    """
    TyDiQA-style preprocessor adapted for UQA (character offsets).
    
    Args:
        examples: Batch with question, context, answer, answer_start fields
        tokenizer: CanineTokenizer instance
        max_length: Maximum sequence length (default 384)
        doc_stride: Sliding window overlap (default 64)
        model_obj: Optional model object (for compatibility)
        indices: Optional example indices for overflow mapping
    
    Returns:
        Dict with input_ids, attention_mask, token_type_ids, start_positions, 
        end_positions, overflow_to_sample_mapping
    """
    questions = [q.strip() for q in examples["question"]]
    contexts = examples["context"]
    answers = examples["answer"]
    answer_starts = examples["answer_start"]
    
    special_tokens = tokenizer.num_special_tokens_to_add(pair=True)
    
    encoded = {
        "input_ids": [],
        "attention_mask": [],
        "token_type_ids": [],
        "start_positions": [],
        "end_positions": [],
        "overflow_to_sample_mapping": [],
    }
    
    for example_idx, (question, context, answer, answer_start) in enumerate(zip(questions, contexts, answers, answer_starts)):
        question_tokens = tokenizer.encode(question, add_special_tokens=False)
        context_tokens = tokenizer.encode(context, add_special_tokens=False)
        
        max_context_tokens = max_length - len(question_tokens) - special_tokens
        if max_context_tokens <= 0 or not context_tokens:
            continue
        
        # UQA uses character offsets (not bytes like TyDiQA)
        if answer and answer_start != -1:
            start_char = answer_start
            end_char = answer_start + len(answer) - 1  # Inclusive
            answer_span = (start_char, end_char)
        else:
            answer_span = None
        
        stride_tokens = max_context_tokens - doc_stride
        if stride_tokens <= 0:
            stride_tokens = max_context_tokens
        
        span_start = 0
        context_length = len(context_tokens)
        while span_start < context_length:
            span_end = min(span_start + max_context_tokens, context_length)
            context_chunk = context_tokens[span_start:span_end]
            
            input_ids = tokenizer.build_inputs_with_special_tokens(question_tokens, context_chunk)
            token_type_ids = tokenizer.create_token_type_ids_from_sequences(question_tokens, context_chunk)
            attention_mask = [1] * len(input_ids)
            
            cls_index = input_ids.index(tokenizer.cls_token_id)
            context_offset = len(input_ids) - len(context_chunk) - 1
            
            if answer_span is None:
                start_pos = cls_index
                end_pos = cls_index
            else:
                start_char, end_char = answer_span
                # CRITICAL FIX: Use < instead of <= for exclusive chunk_end
                answer_in_chunk = start_char >= span_start and end_char < span_end
                if answer_in_chunk:
                    start_pos = context_offset + (start_char - span_start)
                    end_pos = context_offset + (end_char - span_start)
                else:
                    start_pos = cls_index
                    end_pos = cls_index
            
            padding = max_length - len(input_ids)
            if padding > 0:
                pad_id = tokenizer.pad_token_id
                input_ids += [pad_id] * padding
                attention_mask += [0] * padding
                token_type_ids += [0] * padding
            else:
                input_ids = input_ids[:max_length]
                attention_mask = attention_mask[:max_length]
                token_type_ids = token_type_ids[:max_length]
                if start_pos >= max_length or end_pos >= max_length:
                    start_pos = cls_index
                    end_pos = cls_index
            
            encoded["input_ids"].append(input_ids)
            encoded["attention_mask"].append(attention_mask)
            encoded["token_type_ids"].append(token_type_ids)
            encoded["start_positions"].append(start_pos)
            encoded["end_positions"].append(end_pos)
            encoded["overflow_to_sample_mapping"].append(example_idx if indices is None else indices[example_idx])
            
            if span_end == context_length:
                break
            span_start += stride_tokens
    
    return encoded


In [190]:
# LoRA config
lora_config = LoraConfig(
    task_type=TaskType.QUESTION_ANS,
    r=8,   # shadowing tydiqa for now
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["query", "value"], # shadowing tydiqa for now
    bias="none",
    modules_to_save=["qa_outputs"],
)

def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")


### Preprocessing examples...

In [191]:

print("="*80)
print("🔬 PREPROCESSING WALKTHROUGH - Single Example")
print("="*80)

# Take one example
example = uqa_train[0]
print(f"\n1️⃣ ORIGINAL DATA")
print("-"*80)
print(f"Question: {example['question']}")
print(f"Answer: '{example['answer']}'")
print(f"Answer position: {example['answer_start']}")
print(f"Context length: {len(example['context'])} characters")

# Preprocess it
batch = {
    'question': [example['question']],
    'context': [example['context']],
    'answer': [example['answer']],
    'answer_start': [example['answer_start']]
}
processed = preprocess_uqa(batch, tokenizer, indices=[0])

print(f"\n2️⃣ AFTER PREPROCESSING")
print("-"*80)
print(f"Number of chunks created: {len(processed['input_ids'])}")
print(f"(Sliding window creates multiple chunks per example)")

# Show first chunk in detail
chunk_idx = 0
print(f"\n3️⃣ CHUNK {chunk_idx} DETAILS")
print("-"*80)
print(f"Input IDs length: {len(processed['input_ids'][chunk_idx])} tokens")
print(f"Start position: {processed['start_positions'][chunk_idx]}")
print(f"End position: {processed['end_positions'][chunk_idx]}")
print(f"Maps to original example: {processed['overflow_to_sample_mapping'][chunk_idx]}")

# Decode the inputs to show what the model sees
input_ids = processed['input_ids'][chunk_idx]
decoded_input = tokenizer.decode(input_ids, skip_special_tokens=False)
print(f"\n4️⃣ DECODED INPUT (first 400 chars, with special tokens)")
print("-"*80)
print(decoded_input[:400] + "...")

# Decode the labeled answer span
start_pos = processed['start_positions'][chunk_idx]
end_pos = processed['end_positions'][chunk_idx]
cls_idx = input_ids.index(tokenizer.cls_token_id) if tokenizer.cls_token_id in input_ids else 0

if start_pos == cls_idx and end_pos == cls_idx:
    labeled_answer = "[NO ANSWER IN THIS CHUNK]"
else:
    labeled_answer = tokenizer.decode(input_ids[start_pos:end_pos+1], skip_special_tokens=True)

print(f"\n5️⃣ LABELED ANSWER SPAN IN THIS CHUNK")
print("-"*80)
print(f"Gold answer: '{example['answer']}'")
print(f"Labeled span: '{labeled_answer}'")
print(f"Match: {labeled_answer.strip() == example['answer'].strip()}")

# Show all chunks for this example
print(f"\n6️⃣ ALL CHUNKS FOR THIS EXAMPLE")
print("-"*80)
for i in range(len(processed['input_ids'])):
    start = processed['start_positions'][i]
    end = processed['end_positions'][i]
    if start == cls_idx and end == cls_idx:
        chunk_answer = "[NO ANSWER]"
    else:
        chunk_answer = tokenizer.decode(processed['input_ids'][i][start:end+1], skip_special_tokens=True).strip()
    has_answer = "✅" if chunk_answer == example['answer'].strip() else "❌"
    print(f"  Chunk {i}: {has_answer} '{chunk_answer[:50]}'")

print("\n" + "="*80)

🔬 PREPROCESSING WALKTHROUGH - Single Example

1️⃣ ORIGINAL DATA
--------------------------------------------------------------------------------
Question: جدید ہوائی جہازوں کی تعمیر میں کیا استعمال کیا جاتا تھا؟
Answer: 'ڈورالومین'
Answer position: 503
Context length: 767 characters

2️⃣ AFTER PREPROCESSING
--------------------------------------------------------------------------------
Number of chunks created: 3
(Sliding window creates multiple chunks per example)

3️⃣ CHUNK 0 DETAILS
--------------------------------------------------------------------------------
Input IDs length: 384 tokens
Start position: 0
End position: 0
Maps to original example: 0

4️⃣ DECODED INPUT (first 400 chars, with special tokens)
--------------------------------------------------------------------------------
جدید ہوائی جہازوں کی تعمیر میں کیا استعمال کیا جاتا تھا؟1906 میں ، الفریڈ وِلم نے بارش سے سخت ہونے والے مرکب دریافت کیے۔ بارش سے سخت ہونے والے مرکب ، جیسے ایلومینیم ، ٹائٹینیم اور تانبے کے کچھ مر

## 🔧 Preprocessing Exploration: Raw Data → Model Input

Now let's see what happens during preprocessing - how we convert text to token IDs and create training labels.

In [192]:
def normalize_answer(text):
    text = (text or "").lower()
    def remove_articles(s):
        return re.sub(r"\b(a|an|the)\b", " ", s)
    def remove_punctuation(s):
        return "".join(ch for ch in s if ch not in string.punctuation)
    def white_space_fix(s):
        return " ".join(s.split())
    return white_space_fix(remove_articles(remove_punctuation(text)))

def exact_match_score(prediction, ground_truth):
    return float(normalize_answer(prediction) == normalize_answer(ground_truth))

def f1_score(prediction, ground_truth):
    pred_tokens = normalize_answer(prediction).split()
    gold_tokens = normalize_answer(ground_truth).split()
    if not gold_tokens:
        return 1.0 if not pred_tokens else 0.0
    if not pred_tokens:
        return 0.0
    common = Counter(pred_tokens) & Counter(gold_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0.0
    precision = num_same / len(pred_tokens)
    recall = num_same / len(gold_tokens)
    # BUGFIX: Prevent division by zero if both precision and recall are 0
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)

def decode_prediction(input_ids, start_idx, end_idx):

    global tokenizer
    
    # Dynamic CLS handling
    cls_index = input_ids.index(tokenizer.cls_token_id) if tokenizer.cls_token_id in input_ids else 0
    
    # No answer case (both point to CLS)
    if start_idx == cls_index and end_idx == cls_index:
        return ""
    
    # Invalid range (start after end) - treat as no answer
    if start_idx > end_idx:
        return ""
    
    # Defensive bounds checking
    if start_idx < 0 or end_idx < 0:
        return ""
    if start_idx >= len(input_ids) or end_idx >= len(input_ids):
        return ""
    
    # Clamp to valid range (additional safety)
    start_idx = max(start_idx, 0)
    end_idx = min(end_idx, len(input_ids) - 1)
    
    # Decode with inclusive slicing [start:end+1]
    text = tokenizer.decode(input_ids[start_idx:end_idx + 1], skip_special_tokens=True)
    return text.strip()

def gold_answer(example):
    if example["answer_start"] == -1:
        return ""
    return example["answer"]

def edit_distance_score(prediction, ground_truth):
    return Levenshtein.ratio(normalize_answer(prediction), normalize_answer(ground_truth))


#--- CHANGED TO MATCH TYDIQA APPROACH
def evaluate_checkpoint(checkpoint_path=None):
    """
    EXACT REPLICA of TyDiQA evaluation approach.
    Loads checkpoint from disk (or uses provided path).
    """
    # Load base model + trained adapter (TyDiQA approach)
    base_model = CanineForQuestionAnswering.from_pretrained(
        model_name, 
        trust_remote_code=False
    )
    
    from peft import PeftModel
    model = PeftModel.from_pretrained(base_model, checkpoint_path)
    model.to(device)
    
    # Exact TyDiQA eval args
    eval_args = TrainingArguments(
        output_dir="outputs/canine-s-uqa-filtered",
        per_device_eval_batch_size=1,  # Match TyDiQA exactly
        dataloader_drop_last=False,
        fp16=False,  # TyDiQA uses False
        bf16=False,
        report_to="none"
    )
    
    eval_trainer = Trainer(
        model=model,
        args=eval_args,
        eval_dataset=processed_val,
        processing_class=tokenizer,  # Use processing_class
    )
    
    # Progress bar (optional, TyDiQA has this)
    print(f"🧪 Evaluating checkpoint: {checkpoint_path}")
    from tqdm.auto import tqdm
    with tqdm(total=len(processed_val), desc="Evaluating", unit="samples") as pbar:
        predictions = eval_trainer.predict(processed_val)
        pbar.update(len(processed_val))
    
    start_logits, end_logits = predictions.predictions
    
    # EXACT TyDiQA aggregation logic
    best_predictions = {}
    for feature_index, feature in enumerate(processed_val):
        sample_idx = int(feature["overflow_to_sample_mapping"])
        input_ids = feature["input_ids"]
        
        start_idx = int(np.argmax(start_logits[feature_index]))
        end_idx = int(np.argmax(end_logits[feature_index]))
        score = float(start_logits[feature_index][start_idx] + end_logits[feature_index][end_idx])
        prediction_text = decode_prediction(input_ids, start_idx, end_idx)
        
        stored = best_predictions.get(sample_idx)
        if stored is None or score > stored[0]:
            best_predictions[sample_idx] = (score, prediction_text)

    # TEST!
    # After best_predictions loop, before computing metrics:
    print(f"\n🔍 Debug: Sample predictions:")
    for idx in list(best_predictions.keys())[:5]:
        score, pred = best_predictions[idx]
        gold = gold_answer(uqa_val[idx])
        print(f"  Pred: '{pred[:50]}' | Gold: '{gold[:50]}'")
    
    # Calculate metrics
    em_scores = []
    f1_scores = []
    edit_dist_scores = []
    for sample_idx, (_, prediction_text) in best_predictions.items():
        reference = gold_answer(uqa_val[int(sample_idx)])
        em_scores.append(exact_match_score(prediction_text, reference))
        f1_scores.append(f1_score(prediction_text, reference))
        edit_dist_scores.append(edit_distance_score(prediction_text, reference))
    
    em = float(np.mean(em_scores)) if em_scores else 0.0
    f1 = float(np.mean(f1_scores)) if f1_scores else 0.0
    edit_dist = float(np.mean(edit_dist_scores)) if edit_dist_scores else 0.0
    
    print(f"Examples evaluated: {len(em_scores)}")
    print(f"Exact Match: {em * 100:.2f}")
    print(f"F1: {f1 * 100:.2f}")
    print(f"Edit Distance (normalized): {edit_dist * 100:.2f}")
    
    return {"exact_match": em, "f1": f1, "edit_distance": edit_dist}

In [193]:
# ⚠️ CRITICAL: Must regenerate preprocessed data with FILTERED dataset
# The old cache was created from unfiltered data - indices won't match!

# print("🔄 Preprocessing filtered dataset (this will take a few minutes)...")
processed_train = uqa_train.map(
    lambda examples, indices: preprocess_uqa(examples, tokenizer, indices=indices), 
    batched=True, 
    remove_columns=uqa_train.column_names, 
    with_indices=True
)
processed_val = uqa_val.map(
    lambda examples, indices: preprocess_uqa(examples, tokenizer, indices=indices), 
    batched=True, 
    remove_columns=uqa_val.column_names, 
    with_indices=True
)

# print(f"✅ Preprocessing complete!")
# print(f"   Training chunks: {len(processed_train):,}")
# print(f"   Validation chunks: {len(processed_val):,}")

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (3179 > 2048). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [194]:
# print("="*80)
# print("📈 DATASET STATISTICS AFTER PREPROCESSING")
# print("="*80)

# # Count chunks per example
# from collections import Counter
# chunks_per_example = Counter(processed_train["overflow_to_sample_mapping"])
# chunks_distribution = Counter(chunks_per_example.values())

# print(f"\n📦 Chunks Distribution:")
# print(f"   Total original examples: {len(uqa_train):,}")
# print(f"   Total preprocessed chunks: {len(processed_train):,}")
# print(f"   Average chunks per example: {len(processed_train)/len(uqa_train):.2f}")
# print(f"\n   Distribution:")
# for num_chunks in sorted(chunks_distribution.keys())[:10]:
#     count = chunks_distribution[num_chunks]
#     print(f"     {num_chunks} chunk(s): {count:,} examples ({count/len(uqa_train)*100:.1f}%)")

# # Count examples with answers in at least one chunk
# examples_with_answers = 0
# for orig_idx in range(len(uqa_train)):
#     # Find all chunks for this example
#     chunk_indices = [i for i, x in enumerate(processed_train["overflow_to_sample_mapping"]) if x == orig_idx]
    
#     # Check if any chunk has an answer (not pointing to CLS)
#     has_answer = False
#     for chunk_idx in chunk_indices:
#         input_ids = processed_train[chunk_idx]["input_ids"]
#         start_pos = processed_train[chunk_idx]["start_positions"]
#         end_pos = processed_train[chunk_idx]["end_positions"]
#         cls_idx = input_ids.index(tokenizer.cls_token_id) if tokenizer.cls_token_id in input_ids else 0
        
#         if not (start_pos == cls_idx and end_pos == cls_idx):
#             has_answer = True
#             break
    
#     if has_answer:
#         examples_with_answers += 1

# print(f"\n✅ Answer Coverage:")
# print(f"   Examples with answer in at least one chunk: {examples_with_answers:,}/{len(uqa_train):,} ({examples_with_answers/len(uqa_train)*100:.1f}%)")
# print(f"   Expected: ~100% (since we filtered impossible questions)")

# print("="*80)

In [195]:
print("="*80)
print("🔍 BOUNDARY LOGIC VERIFICATION")
print("="*80)

# Test the critical boundary check logic
# Find examples where answer is near chunk boundaries

boundary_cases_found = 0
boundary_cases_correct = 0

for proc_idx in random.sample(range(len(processed_train)), min(500, len(processed_train))):
    proc_example = processed_train[proc_idx]
    orig_idx = proc_example["overflow_to_sample_mapping"]
    orig_example = uqa_train[orig_idx]
    
    input_ids = proc_example["input_ids"]
    start_pos = proc_example["start_positions"]
    end_pos = proc_example["end_positions"]
    
    cls_idx = input_ids.index(tokenizer.cls_token_id) if tokenizer.cls_token_id in input_ids else 0
    
    # Skip no-answer cases
    if start_pos == cls_idx and end_pos == cls_idx:
        continue
    
    # Check if this is a boundary case (answer near end of chunk)
    # Context starts after first SEP token
    sep_indices = [k for k, x in enumerate(input_ids) if x == tokenizer.sep_token_id]
    if not sep_indices:
        continue
    
    context_start = sep_indices[0] + 1
    # Find context end (before padding or second SEP)
    try:
        context_end = sep_indices[1] if len(sep_indices) > 1 else len(input_ids)
    except:
        context_end = len(input_ids)
    
    # Check if answer ends near chunk boundary (within last 10 tokens)
    if context_end - end_pos <= 10:
        boundary_cases_found += 1
        
        # Verify the answer is correct
        predicted_answer = tokenizer.decode(input_ids[start_pos:end_pos+1], skip_special_tokens=True).strip()
        gold_ans = orig_example["answer"].strip()
        
        if predicted_answer == gold_ans:
            boundary_cases_correct += 1

print(f"\n📊 Boundary cases found: {boundary_cases_found}")
if boundary_cases_found > 0:
    print(f"✅ Boundary cases correct: {boundary_cases_correct}/{boundary_cases_found} ({boundary_cases_correct/boundary_cases_found*100:.1f}%)")
    print(f"\n💡 This verifies the fix: using `<` instead of `<=` for chunk boundaries")
else:
    print(f"⚠️  No boundary cases found in sample (may need more examples)")

print("="*80)

🔍 BOUNDARY LOGIC VERIFICATION

📊 Boundary cases found: 3
✅ Boundary cases correct: 3/3 (100.0%)

💡 This verifies the fix: using `<` instead of `<=` for chunk boundaries


In [196]:
import random

print("="*80)
print("🧪 VERIFICATION TEST: Preprocessor Correctness")
print("="*80)

# Test on 100 random examples
num_test_samples = 100
test_indices = random.sample(range(len(processed_train)), min(num_test_samples, len(processed_train)))

passed = 0
failed = 0
failed_examples = []

for proc_idx in test_indices:
    proc_example = processed_train[proc_idx]
    orig_idx = proc_example["overflow_to_sample_mapping"]
    orig_example = uqa_train[orig_idx]
    
    input_ids = proc_example["input_ids"]
    start_pos = proc_example["start_positions"]
    end_pos = proc_example["end_positions"]
    
    # Find CLS position
    cls_idx = input_ids.index(tokenizer.cls_token_id) if tokenizer.cls_token_id in input_ids else 0
    
    # Extract predicted answer
    if start_pos == cls_idx and end_pos == cls_idx:
        predicted_answer = ""
    else:
        predicted_answer = tokenizer.decode(input_ids[start_pos:end_pos+1], skip_special_tokens=True).strip()
    
    gold_ans = orig_example["answer"].strip()
    
    # Check if they match
    if predicted_answer == gold_ans or (not gold_ans and start_pos == cls_idx):
        passed += 1
    else:
        failed += 1
        if len(failed_examples) < 5:  # Store first 5 failures for inspection
            failed_examples.append({
                "question": orig_example["question"][:50] + "...",
                "gold": gold_ans,
                "predicted": predicted_answer,
                "positions": f"[{start_pos}, {end_pos}]"
            })

print(f"\n📊 RESULTS:")
print(f"✅ Passed: {passed}/{num_test_samples} ({passed/num_test_samples*100:.1f}%)")
print(f"❌ Failed: {failed}/{num_test_samples} ({failed/num_test_samples*100:.1f}%)")

if failed > 0 and failed_examples:
    print(f"\n⚠️  First {len(failed_examples)} failures:")
    for i, ex in enumerate(failed_examples, 1):
        print(f"\n  Example {i}:")
        print(f"    Question: {ex['question']}")
        print(f"    Expected: '{ex['gold']}'")
        print(f"    Got: '{ex['predicted']}'")
        print(f"    Positions: {ex['positions']}")
else:
    print(f"\n🎉 All examples passed! Preprocessor is working correctly.")

print("="*80)

🧪 VERIFICATION TEST: Preprocessor Correctness

📊 RESULTS:
✅ Passed: 61/100 (61.0%)
❌ Failed: 39/100 (39.0%)

⚠️  First 5 failures:

  Example 1:
    Question: اگر صارفین اصل پلے اسٹیشن گیمز ڈاؤن لوڈ کرنا چاہتے...
    Expected: 'پلی اسٹیشن اسٹور'
    Got: ''
    Positions: [0, 0]

  Example 2:
    Question: محی الدین نے 1967 میں نصیر کی غلطیوں کی وجہ کیا بت...
    Expected: 'ذیابیطس'
    Got: ''
    Positions: [0, 0]

  Example 3:
    Question: رنچو وسٹوسو کس شہر میں واقع ہے؟...
    Expected: 'اورو ویلی'
    Got: ''
    Positions: [0, 0]

  Example 4:
    Question: فینبرگ اسکول آف میڈیسن کے کس گریجویٹ کے نام پر روز...
    Expected: 'میری ہیرس تھامسن'
    Got: ''
    Positions: [0, 0]

  Example 5:
    Question: اسٹوری بورڈز کو کیا نہیں سمجھا جاتا ہے؟...
    Expected: 'حتمی مصنوع'
    Got: ''
    Positions: [0, 0]


## ✅ Verification: Test Preprocessed Results

Before training, let's verify that the new preprocessor produces correct results.

In [197]:
print("="*80)
print("🧪 TEST 1: Training Data Integrity")
print("="*80)

# Verify training data format
print("\n1️⃣ Checking training dataset structure...")
required_columns = ["input_ids", "attention_mask", "token_type_ids", "start_positions", "end_positions", "overflow_to_sample_mapping"]
missing = [col for col in required_columns if col not in processed_train.column_names]

if missing:
    print(f"❌ CRITICAL: Missing columns: {missing}")
else:
    print(f"✅ All required columns present: {required_columns}")

# Check shapes and ranges
print("\n2️⃣ Validating tensor shapes and ranges...")
issues = []

for i in range(min(100, len(processed_train))):
    example = processed_train[i]
    
    # Check lengths
    if len(example["input_ids"]) != MAX_SEQ_LENGTH:
        issues.append(f"Example {i}: input_ids length {len(example['input_ids'])} != {MAX_SEQ_LENGTH}")
    if len(example["attention_mask"]) != MAX_SEQ_LENGTH:
        issues.append(f"Example {i}: attention_mask length mismatch")
    if len(example["token_type_ids"]) != MAX_SEQ_LENGTH:
        issues.append(f"Example {i}: token_type_ids length mismatch")
    
    # Check position ranges
    start = example["start_positions"]
    end = example["end_positions"]
    if start < 0 or start >= MAX_SEQ_LENGTH:
        issues.append(f"Example {i}: start_position {start} out of range")
    if end < 0 or end >= MAX_SEQ_LENGTH:
        issues.append(f"Example {i}: end_position {end} out of range")
    if start > end:
        issues.append(f"Example {i}: start {start} > end {end}")

if issues:
    print(f"❌ Found {len(issues)} issues:")
    for issue in issues[:10]:  # Show first 10
        print(f"   {issue}")
else:
    print(f"✅ All shapes and ranges valid (checked 100 examples)")

# Check overflow mapping
print("\n3️⃣ Validating overflow_to_sample_mapping...")
max_orig_idx = max(processed_train["overflow_to_sample_mapping"])
if max_orig_idx >= len(uqa_train):
    print(f"❌ CRITICAL: overflow_to_sample_mapping has index {max_orig_idx} >= dataset size {len(uqa_train)}")
else:
    print(f"✅ overflow_to_sample_mapping valid (max={max_orig_idx}, dataset size={len(uqa_train)})")

print("\n" + "="*80)

🧪 TEST 1: Training Data Integrity

1️⃣ Checking training dataset structure...
✅ All required columns present: ['input_ids', 'attention_mask', 'token_type_ids', 'start_positions', 'end_positions', 'overflow_to_sample_mapping']

2️⃣ Validating tensor shapes and ranges...
✅ All shapes and ranges valid (checked 100 examples)

3️⃣ Validating overflow_to_sample_mapping...
✅ overflow_to_sample_mapping valid (max=19999, dataset size=20000)



In [198]:
import random
from collections import defaultdict

print("="*80)
print("🧪 FIXED TEST: Answer Extraction Accuracy (OPTIMIZED)")
print("="*80)

# Step 1: Build reverse index ONCE (O(n) instead of O(n²))
print("Building chunk index...")
chunk_index = defaultdict(list)
for chunk_idx, sample_idx in enumerate(processed_train["overflow_to_sample_mapping"]):
    chunk_index[sample_idx].append(chunk_idx)

# Step 2: Test on random original examples
num_samples = 200
test_orig_indices = random.sample(range(len(uqa_train)), num_samples)

correct = 0
incorrect = 0
failed_examples = []

for orig_idx in test_orig_indices:
    orig_example = uqa_train[orig_idx]
    gold_ans = orig_example["answer"].strip()
    
    # Get all chunks for this example (O(1) lookup!)
    chunk_indices = chunk_index[orig_idx]
    
    # Check if ANY chunk has the correct answer
    found_correct = False
    for chunk_idx in chunk_indices:
        proc = processed_train[chunk_idx]
        input_ids = proc["input_ids"]
        start = proc["start_positions"]
        end = proc["end_positions"]
        
        cls_idx = input_ids.index(tokenizer.cls_token_id)
        
        # Skip chunks without answer
        if start == cls_idx and end == cls_idx:
            continue
        
        # Extract answer
        predicted = tokenizer.decode(input_ids[start:end+1], skip_special_tokens=True).strip()
        
        if predicted.lower() == gold_ans.lower():
            found_correct = True
            break
    
    if found_correct or not gold_ans:
        correct += 1
    else:
        incorrect += 1
        if len(failed_examples) < 5:
            failed_examples.append({
                "idx": orig_idx,
                "question": orig_example["question"][:60],
                "gold": gold_ans[:50],
                "num_chunks": len(chunk_indices)
            })

accuracy = correct / num_samples * 100
print(f"\n📊 Results: ✅ {correct}/{num_samples} ({accuracy:.1f}%)")

if accuracy >= 95:
    print("✅ PASSED - Preprocessor working correctly!")
else:
    print(f"❌ FAILED - Only {accuracy:.1f}% accuracy")
    if failed_examples:
        print(f"\n⚠️ First {len(failed_examples)} failures:")
        for ex in failed_examples:
            print(f"  #{ex['idx']}: '{ex['question']}...'")
            print(f"    Expected: '{ex['gold']}', Chunks: {ex['num_chunks']}")

print("="*80)

🧪 FIXED TEST: Answer Extraction Accuracy (OPTIMIZED)
Building chunk index...

📊 Results: ✅ 200/200 (100.0%)
✅ PASSED - Preprocessor working correctly!


In [199]:
print("="*80)
print("🧪 TEST 3: Validation Data Integrity")
print("="*80)

# Same checks for validation data
print("\n1️⃣ Checking validation dataset structure...")
missing_val = [col for col in required_columns if col not in processed_val.column_names]

if missing_val:
    print(f"❌ CRITICAL: Missing columns: {missing_val}")
else:
    print(f"✅ All required columns present")

# Check validation mapping
print("\n2️⃣ Validating overflow_to_sample_mapping...")
max_val_idx = max(processed_val["overflow_to_sample_mapping"])
if max_val_idx >= len(uqa_val):
    print(f"❌ CRITICAL: overflow_to_sample_mapping has index {max_val_idx} >= dataset size {len(uqa_val)}")
else:
    print(f"✅ overflow_to_sample_mapping valid (max={max_val_idx}, dataset size={len(uqa_val)})")

# Test extraction on validation
print("\n3️⃣ Testing answer extraction on validation set...")
val_correct = 0
val_incorrect = 0
val_samples = min(100, len(processed_val))

for proc_idx in range(val_samples):
    proc_example = processed_val[proc_idx]
    orig_idx = proc_example["overflow_to_sample_mapping"]
    orig_example = uqa_val[orig_idx]
    
    input_ids = proc_example["input_ids"]
    start_pos = proc_example["start_positions"]
    end_pos = proc_example["end_positions"]
    
    cls_idx = input_ids.index(tokenizer.cls_token_id) if tokenizer.cls_token_id in input_ids else 0
    
    if start_pos == cls_idx and end_pos == cls_idx:
        predicted_answer = ""
    else:
        predicted_answer = tokenizer.decode(input_ids[start_pos:end_pos+1], skip_special_tokens=True).strip()
    
    gold_ans = orig_example["answer"].strip()

    print(f"GOLD: {gold_ans.lower()}")
    print(f"PREDICTED: {predicted_answer.lower()}\n")
    
    if predicted_answer.lower() == gold_ans.lower():
        val_correct += 1
    else:
        val_incorrect += 1

val_accuracy = val_correct / val_samples * 100
print(f"   Validation accuracy: {val_correct}/{val_samples} ({val_accuracy:.1f}%)")

if val_accuracy < 95:
    print(f"   ❌ WARNING: Validation accuracy is low!")
else:
    print(f"   ✅ Validation data is correct!")

print("="*80)

🧪 TEST 3: Validation Data Integrity

1️⃣ Checking validation dataset structure...
✅ All required columns present

2️⃣ Validating overflow_to_sample_mapping...
✅ overflow_to_sample_mapping valid (max=4999, dataset size=5000)

3️⃣ Testing answer extraction on validation set...
GOLD: مغربی کیپ صوبہ
PREDICTED: مغربی کیپ صوبہ

GOLD: مغربی کیپ صوبہ
PREDICTED: 

GOLD: مغربی کیپ صوبہ
PREDICTED: 

GOLD: 
PREDICTED: 

GOLD: 
PREDICTED: 

GOLD: 
PREDICTED: 

GOLD: 
PREDICTED: 

GOLD: 
PREDICTED: 

GOLD: ہیوگس مفروضہ
PREDICTED: ہیوگس مفروضہ

GOLD: ہیوگس مفروضہ
PREDICTED: 

GOLD: 
PREDICTED: 

GOLD: 
PREDICTED: 

GOLD: 
PREDICTED: 

GOLD: 
PREDICTED: 

GOLD: 
PREDICTED: 

GOLD: گالش نام ریونوس
PREDICTED: گالش نام ریونوس

GOLD: گالش نام ریونوس
PREDICTED: 

GOLD: گالش نام ریونوس
PREDICTED: 

GOLD: p
PREDICTED: 

GOLD: p
PREDICTED: p

GOLD: p
PREDICTED: 

GOLD: نیٹ ورکنگ فوائد کو بڑھانا
PREDICTED: نیٹ ورکنگ فوائد کو بڑھانا

GOLD: نیٹ ورکنگ فوائد کو بڑھانا
PREDICTED: 

GOLD: تقریبا 20 گھنٹوں
PREDICTED:

### !!! KEY TAKEAWAY FROM ABOVE CELLS!

A lot of chunks do not have the answer in the chunked context, so (0, 0) -> `[CLS]` tok is being predicted!
This may be giving way to a lot of mispredictions in evaluation!

In [200]:
print("="*80)
print("🧪 TEST 4: Evaluation Functions Correctness")
print("="*80)

# Test the metric functions
print("\n1️⃣ Testing normalize_answer()...")
test_cases = [
    ("Hello World", "hello world"),
    ("The quick fox", "quick fox"),
    ("Test!", "test"),
    ("  spaces  ", "spaces"),
]

for input_text, expected in test_cases:
    result = normalize_answer(input_text)
    status = "✅" if result == expected else "❌"
    print(f"   {status} normalize_answer('{input_text}') = '{result}' (expected: '{expected}')")

# Test exact_match_score
print("\n2️⃣ Testing exact_match_score()...")
em_tests = [
    ("hello", "hello", 1.0),
    ("hello", "Hello", 1.0),  # Case insensitive
    ("the answer", "answer", 1.0),  # Articles removed
    ("hello", "world", 0.0),
    ("", "", 1.0),
]

for pred, gold, expected in em_tests:
    result = exact_match_score(pred, gold)
    status = "✅" if result == expected else "❌"
    print(f"   {status} EM('{pred}', '{gold}') = {result} (expected: {expected})")

# Test f1_score
print("\n3️⃣ Testing f1_score()...")
f1_tests = [
    ("hello world", "hello world", 1.0),
    ("hello", "world", 0.0),
    ("hello world", "hello", 0.67),  # Approximate
    ("", "", 1.0),
    ("hello", "", 0.0),
]

all_f1_ok = True
for pred, gold, expected in f1_tests:
    result = f1_score(pred, gold)
    # Allow small tolerance for floating point
    ok = abs(result - expected) < 0.01 or (expected == 0 and result == 0)
    status = "✅" if ok else "❌"
    if not ok:
        all_f1_ok = False
    print(f"   {status} F1('{pred}', '{gold}') = {result:.2f} (expected: ~{expected})")

# Test decode_prediction
print("\n4️⃣ Testing decode_prediction()...")
sample_ids = tokenizer.encode("This is a test answer", add_special_tokens=True)
cls_idx = sample_ids.index(tokenizer.cls_token_id)

decode_tests = [
    (sample_ids, cls_idx, cls_idx, ""),  # No answer case
    (sample_ids, 5, 3, ""),  # Invalid range (start > end)
    (sample_ids, -1, 5, ""),  # Negative index
    (sample_ids, 2, 5, "non-empty"),  # Valid range should return something
]

for ids, start, end, expected_type in decode_tests:
    result = decode_prediction(ids, start, end)
    if expected_type == "":
        ok = result == ""
        status = "✅" if ok else "❌"
        print(f"   {status} decode_prediction(..., {start}, {end}) = '{result}' (expected empty)")
    else:
        ok = len(result) > 0
        status = "✅" if ok else "❌"
        print(f"   {status} decode_prediction(..., {start}, {end}) = '{result}' (expected non-empty)")

print("\n" + "="*80)

🧪 TEST 4: Evaluation Functions Correctness

1️⃣ Testing normalize_answer()...
   ✅ normalize_answer('Hello World') = 'hello world' (expected: 'hello world')
   ✅ normalize_answer('The quick fox') = 'quick fox' (expected: 'quick fox')
   ✅ normalize_answer('Test!') = 'test' (expected: 'test')
   ✅ normalize_answer('  spaces  ') = 'spaces' (expected: 'spaces')

2️⃣ Testing exact_match_score()...
   ✅ EM('hello', 'hello') = 1.0 (expected: 1.0)
   ✅ EM('hello', 'Hello') = 1.0 (expected: 1.0)
   ✅ EM('the answer', 'answer') = 1.0 (expected: 1.0)
   ✅ EM('hello', 'world') = 0.0 (expected: 0.0)
   ✅ EM('', '') = 1.0 (expected: 1.0)

3️⃣ Testing f1_score()...
   ✅ F1('hello world', 'hello world') = 1.00 (expected: ~1.0)
   ✅ F1('hello', 'world') = 0.00 (expected: ~0.0)
   ✅ F1('hello world', 'hello') = 0.67 (expected: ~0.67)
   ✅ F1('', '') = 1.00 (expected: ~1.0)
   ✅ F1('hello', '') = 0.00 (expected: ~0.0)

4️⃣ Testing decode_prediction()...
   ✅ decode_prediction(..., 0, 0) = '' (expected e

In [201]:
print("="*80)
print("🧪 TEST 5: Model Forward Pass (Sanity Check)")
print("="*80)

# Test that model can process a batch
print("\n1️⃣ Testing model forward pass...")

try:
    # Take a small batch
    batch_size = 4
    sample_batch = processed_train.select(range(batch_size))
    
    # Convert to tensors
    input_ids = torch.tensor(sample_batch["input_ids"]).to(device)
    attention_mask = torch.tensor(sample_batch["attention_mask"]).to(device)
    token_type_ids = torch.tensor(sample_batch["token_type_ids"]).to(device)
    start_positions = torch.tensor(sample_batch["start_positions"]).to(device)
    end_positions = torch.tensor(sample_batch["end_positions"]).to(device)
    
    print(f"   Input shape: {input_ids.shape}")
    print(f"   Attention mask shape: {attention_mask.shape}")
    print(f"   Token type IDs shape: {token_type_ids.shape}")
    
    # Forward pass
    model.to(device)
    model.eval()
    
    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            start_positions=start_positions,
            end_positions=end_positions
        )
    
    print(f"\n   ✅ Forward pass successful!")
    print(f"   Loss: {outputs.loss.item():.4f}")
    print(f"   Start logits shape: {outputs.start_logits.shape}")
    print(f"   End logits shape: {outputs.end_logits.shape}")
    
    # Check logits are valid
    if torch.isnan(outputs.start_logits).any() or torch.isnan(outputs.end_logits).any():
        print(f"   ❌ WARNING: NaN values in logits!")
    else:
        print(f"   ✅ Logits are valid (no NaN)")
    
    # Check loss is reasonable
    if outputs.loss.item() < 0 or outputs.loss.item() > 100:
        print(f"   ⚠️  WARNING: Loss seems unusual: {outputs.loss.item()}")
    else:
        print(f"   ✅ Loss is in reasonable range")
    
except Exception as e:
    print(f"   ❌ CRITICAL ERROR during forward pass: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "="*80)

🧪 TEST 5: Model Forward Pass (Sanity Check)

1️⃣ Testing model forward pass...
   Input shape: torch.Size([4, 384])
   Attention mask shape: torch.Size([4, 384])
   Token type IDs shape: torch.Size([4, 384])

   ✅ Forward pass successful!
   Loss: 5.9048
   Start logits shape: torch.Size([4, 384])
   End logits shape: torch.Size([4, 384])
   ✅ Logits are valid (no NaN)
   ✅ Loss is in reasonable range



In [202]:
print("="*80)
print("🧪 TEST 6: Critical Boundary Cases")
print("="*80)

# Verify the fix for the <= vs < bug
print("\n1️⃣ Testing chunk boundary logic (the critical bug fix)...")

boundary_correct = 0
boundary_total = 0

for proc_idx in range(min(1000, len(processed_train))):
    proc_example = processed_train[proc_idx]
    orig_idx = proc_example["overflow_to_sample_mapping"]
    orig_example = uqa_train[orig_idx]
    
    input_ids = proc_example["input_ids"]
    start_pos = proc_example["start_positions"]
    end_pos = proc_example["end_positions"]
    
    cls_idx = input_ids.index(tokenizer.cls_token_id) if tokenizer.cls_token_id in input_ids else 0
    
    # Skip no-answer cases
    if start_pos == cls_idx:
        continue
    
    # Find context boundaries
    sep_indices = [k for k, x in enumerate(input_ids) if x == tokenizer.sep_token_id]
    if not sep_indices:
        continue
    
    context_start = sep_indices[0] + 1
    
    # Check if answer is near end of context chunk (within last 5 positions)
    # This is where the bug would manifest
    if len(sep_indices) > 1:
        context_end = sep_indices[1]
    else:
        # Find first padding token
        context_end = next((i for i, x in enumerate(input_ids) if x == tokenizer.pad_token_id), len(input_ids))
    
    if context_end - end_pos <= 5:
        boundary_total += 1
        
        # Verify extraction is correct
        predicted = tokenizer.decode(input_ids[start_pos:end_pos+1], skip_special_tokens=True).strip()
        gold = orig_example["answer"].strip()
        
        if predicted.lower() == gold.lower():
            boundary_correct += 1

print(f"\n   Found {boundary_total} boundary cases (answer near chunk end)")
if boundary_total > 0:
    boundary_accuracy = boundary_correct / boundary_total * 100
    print(f"   Boundary cases correct: {boundary_correct}/{boundary_total} ({boundary_accuracy:.1f}%)")
    
    if boundary_accuracy < 95:
        print(f"   ❌ WARNING: Boundary logic may still have issues!")
    else:
        print(f"   ✅ Boundary fix is working correctly!")
else:
    print(f"   ⚠️  No boundary cases found in first 1000 examples")

# Test the specific case from verification script
print("\n2️⃣ Testing the exact bug scenario...")
# Answer [90, 99] inclusive, Chunk [0, 100) exclusive
test_start = 90
test_end = 99  # inclusive
chunk_start = 0
chunk_end = 100  # exclusive

# Correct logic (what we implemented)
correct_result = test_start >= chunk_start and test_end < chunk_end
# Buggy logic (what we fixed)
buggy_result = test_start >= chunk_start and test_end <= chunk_end

print(f"   Scenario: answer=[{test_start},{test_end}], chunk=[{chunk_start},{chunk_end})")
print(f"   Correct logic (< for end): {correct_result}")
print(f"   Buggy logic (<= for end): {buggy_result}")

if correct_result == True and buggy_result == True:
    print(f"   ✅ Both agree when answer is inside chunk")
elif correct_result != buggy_result:
    print(f"   ⚠️  Logics differ - this is where the bug would cause mislabeling")

# Now test the failing case
test_end = 100  # Now extends beyond
correct_result = test_start >= chunk_start and test_end < chunk_end
buggy_result = test_start >= chunk_start and test_end <= chunk_end

print(f"\n   Scenario: answer=[{test_start},{test_end}], chunk=[{chunk_start},{chunk_end})")
print(f"   Correct logic (< for end): {correct_result} ✅")
print(f"   Buggy logic (<= for end): {buggy_result} ❌")

if correct_result == False and buggy_result == True:
    print(f"   ✅ Fix verified: correct logic rejects, buggy logic accepts (WRONG)")
else:
    print(f"   ❌ Something is wrong with the logic")

print("\n" + "="*80)

🧪 TEST 6: Critical Boundary Cases

1️⃣ Testing chunk boundary logic (the critical bug fix)...

   Found 2 boundary cases (answer near chunk end)
   Boundary cases correct: 2/2 (100.0%)
   ✅ Boundary fix is working correctly!

2️⃣ Testing the exact bug scenario...
   Scenario: answer=[90,99], chunk=[0,100)
   Correct logic (< for end): True
   Buggy logic (<= for end): True
   ✅ Both agree when answer is inside chunk

   Scenario: answer=[90,100], chunk=[0,100)
   Correct logic (< for end): False ✅
   Buggy logic (<= for end): True ❌
   ✅ Fix verified: correct logic rejects, buggy logic accepts (WRONG)



---

## 🔬 COMPREHENSIVE QA PIPELINE VERIFICATION

Before training, let's verify **every single component** of the QA pipeline end-to-end.

In [203]:
# processed_train

In [204]:
# processed_val

In [205]:
# Save newly processed data (OPTIONAL - for future reuse with same filtered dataset)
processed_train.save_to_disk("/kaggle/working/cache/processed_train_uqa_filtered")
processed_val.save_to_disk("/kaggle/working/cache/processed_val_uqa_filtered")

# # ❌ DO NOT load old cache - it has index mismatches with filtered data!
# # If you've already run the preprocessing cell above, skip this cell

processed_train = load_from_disk("/kaggle/working/cache/processed_train_uqa_filtered")
processed_val = load_from_disk("/kaggle/working/cache/processed_val_uqa_filtered")

Saving the dataset (0/1 shards):   0%|          | 0/58369 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/15786 [00:00<?, ? examples/s]

In [206]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")



In [207]:
# build LoRA model

peft_model = get_peft_model(model, lora_config)
peft_model.gradient_checkpointing_enable()
print_trainable_parameters(peft_model)

trainable params: 345602 || all params: 132430084 || trainable%: 0.26096940329661045


In [208]:
# Show what the model sees during training
print("="*80)
print("🎓 MODEL TRAINING DATA FLOW")
print("="*80)

# Take one batch from preprocessed data
batch_size = 4
sample_batch = processed_train.select(range(batch_size))

print(f"\n1️⃣ BATCH STRUCTURE")
print("-"*80)
print(f"Batch size: {batch_size} chunks")
print(f"Each chunk in the batch contains:")

# Show batch structure
for key in sample_batch.column_names:
    sample_value = sample_batch[0][key]
    if isinstance(sample_value, list):
        print(f"  - {key}: shape ({batch_size}, {len(sample_value)})")
    else:
        print(f"  - {key}: shape ({batch_size},)")

print(f"\n2️⃣ WHAT THE MODEL RECEIVES (for 1 chunk in batch)")
print("-"*80)
example_idx = 0
print(f"Input IDs: {len(sample_batch[example_idx]['input_ids'])} tokens")
print(f"  First 10 token IDs: {sample_batch[example_idx]['input_ids'][:10]}")
print(f"\nAttention mask: {sample_batch[example_idx]['attention_mask'][:20]}...")
print(f"  (1=attend to token, 0=ignore padding)")
print(f"\nToken type IDs: {sample_batch[example_idx]['token_type_ids'][:20]}...")
print(f"  (0=question tokens, 1=context tokens)")

print(f"\n3️⃣ TRAINING TARGETS (what model learns to predict)")
print("-"*80)
print(f"Target start position: {sample_batch[example_idx]['start_positions']}")
print(f"Target end position: {sample_batch[example_idx]['end_positions']}")
print(f"\n💡 The model learns to output these exact positions!")

print("\n" + "="*80)

🎓 MODEL TRAINING DATA FLOW

1️⃣ BATCH STRUCTURE
--------------------------------------------------------------------------------
Batch size: 4 chunks
Each chunk in the batch contains:
  - input_ids: shape (4, 384)
  - attention_mask: shape (4, 384)
  - token_type_ids: shape (4, 384)
  - start_positions: shape (4,)
  - end_positions: shape (4,)
  - overflow_to_sample_mapping: shape (4,)

2️⃣ WHAT THE MODEL RECEIVES (for 1 chunk in batch)
--------------------------------------------------------------------------------
Input IDs: 384 tokens
  First 10 token IDs: [57344, 1580, 1583, 1740, 1583, 32, 1729, 1608, 1575, 1574]

Attention mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]...
  (1=attend to token, 0=ignore padding)

Token type IDs: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...
  (0=question tokens, 1=context tokens)

3️⃣ TRAINING TARGETS (what model learns to predict)
-------------------------------------------------------------------------------

---

## Model Training:


In [209]:
training_args = TrainingArguments(
    output_dir="outputs/canine-s-uqa-filtered",

    per_device_train_batch_size=4,  # increased train_batch_size from tydiqa
    per_device_eval_batch_size=16,

    gradient_accumulation_steps=4,  # decreased grad accum from tydiqa
    gradient_checkpointing=True,

    num_train_epochs=1, # same as tydiqa
    learning_rate=3e-5,  
    weight_decay=0.01,
    
    eval_strategy="no",
    eval_steps=500,
    save_strategy="steps",
    save_steps=1000,  # increased to 1000
    logging_steps=50,
    
    fp16=True,
    bf16=False,
    report_to="none",
    push_to_hub=True,
    hub_model_id="VohraAK/canine-s-uqa-filtered",
    hub_strategy="checkpoint",
    )

# CustomEvalCallback - EXACT TyDiQA approach
class CustomEvalCallback(TrainerCallback):
    def __init__(self, eval_func, eval_dataset):
        self.eval_func = eval_func
        self.eval_dataset = eval_dataset

    def on_save(self, args, state, control, model=None, **kwargs):
        """
        Runs AFTER checkpoint is saved.
        Loads checkpoint from disk and evaluates it.
        """
        checkpoint_path = f"{args.output_dir}/checkpoint-{state.global_step}"
        print(f"\n🔍 Running custom evaluation at step {state.global_step}...")

        # Call evaluation function (loads from checkpoint)
        metrics = self.eval_func(checkpoint_path)

        # Add metrics to state's log_history
        state.log_history.append({
            "step": state.global_step,
            "eval_exact_match": metrics["exact_match"],
            "eval_f1": metrics["f1"],
            "eval_edit_distance": metrics["edit_distance"],
        })

        # Print metrics
        print(f"✅ Step {state.global_step}: EM={metrics['exact_match']*100:.2f}, F1={metrics['f1']*100:.2f}, EditDist={metrics['edit_distance']*100:.2f}")

        # Re-save trainer_state.json with updated metrics
        state_path = f"{checkpoint_path}/trainer_state.json"
        try:
            with open(state_path, 'r') as f:
                state_dict = json.load(f)
            state_dict['log_history'] = state.log_history
            with open(state_path, 'w') as f:
                json.dump(state_dict, f, indent=2)
            print(f"💾 Updated trainer_state.json with custom metrics")
        except Exception as e:
            print(f"⚠️  Warning: Could not update trainer_state.json: {e}")

        # Push to Hub
        try:
            print(f"☁️  Pushing checkpoint-{state.global_step} to Hub...")
            api = HfApi()
            api.upload_folder(
                folder_path=checkpoint_path,
                repo_id=args.hub_model_id,
                path_in_repo=f"checkpoint-{state.global_step}",
                commit_message=f"Add checkpoint {state.global_step} (EM={metrics['exact_match']*100:.1f}%, F1={metrics['f1']*100:.1f}%)",
                repo_type="model"
            )
            print(f"✅ Pushed checkpoint-{state.global_step} to Hub")
        except Exception as e:
            print(f"⚠️  Warning: Could not push to Hub: {e}")

        return control



In [210]:
trainer_cb = CustomEvalCallback(evaluate_checkpoint, processed_val)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=processed_train,
    eval_dataset=processed_val,
    callbacks=[trainer_cb],
)

No label_names provided for model class `PeftModelForQuestionAnswering`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
50,5.939700
100,5.894100
150,5.843800
200,5.809200
250,5.752200
300,5.720200
350,5.680400
400,5.648900
450,5.599100
500,5.580700



🔍 Running custom evaluation at step 1000...


Some weights of CanineForQuestionAnswering were not initialized from the model checkpoint at google/canine-s and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForQuestionAnswering`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


🧪 Evaluating checkpoint: outputs/canine-s-uqa-filtered/checkpoint-1000


...

/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



🔍 Running custom evaluation at step 3000...


Some weights of CanineForQuestionAnswering were not initialized from the model checkpoint at google/canine-s and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForQuestionAnswering`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


🧪 Evaluating checkpoint: outputs/canine-s-uqa-filtered/checkpoint-3000


Evaluating:   0%|          | 0/15786 [00:00<?, ?samples/s]

---

### Diagnosing Preprocessing Functions!!!

These functions are just analysing the preprocessing logic above, they're just using the base model, NOT our trained model...

In [ ]:
# # Diagnostic cell (fixed): Investigate preprocessing and truncation for many samples
# import random
# import pandas as pd
# from transformers import AutoTokenizer

# # Set display options to see full Urdu text
# pd.set_option('display.max_colwidth', None)

# try:
#     tokenizer = AutoTokenizer.from_pretrained("google/canine-s")
# except Exception:
#     tokenizer = None

# num_samples = 20000  # Number of samples to check
# results = []

# for split_name, orig_data, proc_data in [
#     ("train", uqa_train, processed_train),
#     ("val", uqa_val, processed_val)
# ]:
#     # Sample random indices
#     if len(proc_data) < num_samples:
#         current_indices = range(len(proc_data))
#     else:
#         current_indices = random.sample(range(len(proc_data)), num_samples)

#     for idx in current_indices:
#         proc = proc_data[idx]
#         # Use overflow_to_sample_mapping to get the correct original index
#         orig_idx = proc["overflow_to_sample_mapping"]
#         orig = orig_data[orig_idx]

#         input_ids = proc["input_ids"]
#         start_pos = proc["start_positions"]
#         end_pos = proc["end_positions"]

#         gold_answer = orig.get("gold_answer", orig.get("answer", ""))
#         question = orig.get("question", "")

#         # Decode input_ids to text (for debugging context)
#         if tokenizer:
#             decoded_text = tokenizer.decode(input_ids, skip_special_tokens=False)
#         else:
#             decoded_text = str(input_ids)

#         # Extract predicted answer span
#         if 0 <= start_pos < len(input_ids) and 0 <= end_pos < len(input_ids):
#             if tokenizer:
#                 pred_span = tokenizer.decode(input_ids[start_pos:end_pos+1], skip_special_tokens=True)
#             else:
#                 pred_span = str(input_ids[start_pos:end_pos+1])
#         else:
#             pred_span = "[CLS]" # Represents no answer found in this chunk or invalid

#         # Check if pred_span matches gold answer
#         # We strip() to ignore minor whitespace differences
#         pred_matches_gold = pred_span.strip() == gold_answer.strip()

#         # Check if gold is even reachable in this chunk
#         gold_in_decoded = gold_answer in decoded_text

#         results.append({
#             "Split": split_name,
#             "Question": question,
#             "Gold Answer": gold_answer,
#             "Extracted Answer": pred_span,
#             "Match": pred_matches_gold,
#             "Gold Reachable": gold_in_decoded,
#             "orig_idx": orig_idx
#         })

# # Create DataFrame
# results_df = pd.DataFrame(results)

# # --- SIDE BY SIDE COMPARISON ---

# # 1. Filter for Solvable Mismatches (Gold was there, but we predicted wrong)
# problem_cases = results_df[
#     (results_df["Gold Reachable"] == True) &
#     (results_df["Match"] == False)
# ][["Question", "Gold Answer", "Extracted Answer", "Split"]]

# print(f"🔍 Checked {len(results_df)} samples.")
# print(f"❌ Found {len(problem_cases)} cases where Gold was present but Extraction failed.")

# print("\n📊 Side-by-Side Comparison (Top 20 Failures):")
# display(problem_cases.head(50))

# print("\n✅ Side-by-Side Comparison (First 10 Rows - Mixed):")
# display(results_df[["Question", "Gold Answer", "Extracted Answer", "Match"]].head(50))

In [ ]:
# # Accuracy: fraction of rows where extracted answer matches gold answer
# accuracy = (results_df["Match"]).mean()

# # Precision: among rows where extracted answer is non-empty, fraction that matches gold
# # We filter out cases where the model predicted nothing (empty string) or just whitespace
# non_empty_pred = results_df["Extracted Answer"].str.strip() != ""

# # Avoid division by zero if no predictions were made
# if non_empty_pred.sum() > 0:
#     precision = (results_df["Match"] & non_empty_pred).sum() / non_empty_pred.sum()
# else:
#     precision = 0.0

# print(f"Accuracy: {accuracy:.3f}")
# print(f"Precision: {precision:.3f}")